In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Base Dataset

与Original数据的分布保持一致，Majority：Minority=77：23（原数据为80：20）
Prevelance：Majority=0.009，Minority=0.018 （原数据为0.01，0.02）；两倍关系。

In [2]:
df = pd.read_csv('datasets/Base.csv')
df

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200.0,0,INTERNET,3.743048,other,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0.8,0.124690,-1,143,30,0.051348,-0.826239,AB,530,...,0,1500.0,0,INTERNET,16.967770,other,0,1,0,7
999996,0,0.9,0.824544,-1,193,30,0.009591,0.008307,AC,408,...,1,1000.0,0,INTERNET,1.504109,macintosh,0,1,0,7
999997,0,0.8,0.140891,-1,202,10,0.059287,50.609995,AA,749,...,0,200.0,0,INTERNET,16.068595,other,0,1,0,7
999998,0,0.9,0.002480,52,3,30,0.023357,-1.313387,AB,707,...,0,200.0,0,INTERNET,1.378683,linux,1,1,0,7


In [3]:
# 查看fraud bool具体情况
df['fraud_bool'].value_counts()

fraud_bool
0    988971
1     11029
Name: count, dtype: int64

In [100]:
# 查看customer_age大于40并且小于80的数量
age50_more=df[df['customer_age']>=50]

# 查看customer_age小于50的数量
age50_less=df[df['customer_age']<=40]

age50_less.shape[0]/(age50_more.shape[0]+age50_less.shape[0])

0.816987

In [108]:
# 查看age50_more的欺诈率（即fraud_bool=1的比例）
prevalence_major=age50_more[age50_more['fraud_bool']==1].shape[0]/age50_more.shape[0]
prevalence_minor=age50_less[age50_less['fraud_bool']==1].shape[0]/age50_less.shape[0]

prevalence_major, prevalence_minor

(0.023413637282597412, 0.008254721311355016)

# 2. Variant I Dataset

该数据集控制了prevalence在两个群体直接保持一致，但是放大了群体比例（90：10）

Majority：Minority=90：10

Prevelance：Majority=0.011，Minority=0.011

In [117]:
df1=pd.read_csv('datasets/Variant I.csv')
df1

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.9,0.990415,11,4,40,0.014640,-0.897718,AC,1853,...,0,200.0,0,INTERNET,6.150317,windows,0,1,0,0
1,0,0.4,0.781728,-1,151,30,0.013269,11.501507,AA,5447,...,0,1500.0,0,INTERNET,4.521952,windows,1,1,0,0
2,0,0.7,0.274290,-1,92,20,0.010669,23.943928,AA,2312,...,0,200.0,0,INTERNET,8.361707,other,1,1,0,0
3,0,0.9,0.722684,-1,55,20,0.021849,48.202774,AA,2413,...,0,1500.0,0,INTERNET,7.339743,other,0,1,0,0
4,0,0.9,0.121661,-1,51,20,0.021491,-1.602116,AB,1227,...,0,1500.0,0,INTERNET,4.338063,other,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0.9,0.173671,-1,110,30,0.004057,-1.451974,AB,979,...,0,200.0,0,INTERNET,5.740367,other,1,1,0,7
999996,0,0.9,0.566285,-1,29,50,0.014215,-0.672890,AB,1190,...,0,200.0,0,INTERNET,26.319296,linux,0,1,0,7
999997,0,0.6,0.807476,-1,164,30,0.040160,46.129201,AA,1571,...,0,1000.0,0,INTERNET,1.614444,linux,1,1,0,7
999998,0,0.8,0.151376,87,12,30,0.016871,-1.212084,AC,1435,...,0,200.0,0,INTERNET,3.682591,linux,0,1,0,7


In [18]:
df1['fraud_bool'].value_counts()

fraud_bool
0    988970
1     11030
Name: count, dtype: int64

In [118]:
# 查看customer_age大于40并且小于80的数量
age50_more=df1[df1['customer_age']>=50]

# 查看customer_age小于50的数量
age50_less=df1[df1['customer_age']<=40]

age50_less.shape[0]/(age50_more.shape[0]+age50_less.shape[0])

0.889301

In [119]:
# 查看age50_more和age50_less的欺诈率（即fraud_bool=1的比例）
prevalence_major=age50_more[age50_more['fraud_bool']==1].shape[0]/age50_more.shape[0]
prevalence_minor=age50_less[age50_less['fraud_bool']==1].shape[0]/age50_less.shape[0]

prevalence_major, prevalence_minor

(0.012159098094833738, 0.010888326899441246)

# 3. Variant III Dataset
存在可分离性差异。对于多数类，更容易区分欺诈和非欺诈，而对于少数类，更难区分欺诈和非欺诈。（使用新特征x1，x2）

Majority：Minority=50：50

Prevelance：Majority=0.0011，Minority=0.0011

Separability：Majority=Increased，Minority=Equal

# 4. Variant IV Dataset
该数据集引入时间偏差，前六个月和后两个月的多数类群体和少数类群体的欺诈率不同（前六个月两个群体欺诈率不一致，后两个月中两个群体欺诈率一致）

Majority：Minority=50：50

Prevelance：Majority=0.003-->0.0015，Minority=0.0017-->0.0015

In [120]:
df4=pd.read_csv('datasets/Variant IV.csv')
df4

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.9,0.356764,27,44,30,0.009697,18.851662,AA,1091,...,1,200.0,0,INTERNET,3.698583,windows,0,1,0,0
1,0,0.6,0.045919,20,0,20,0.029158,-0.977631,AB,2320,...,0,200.0,0,INTERNET,9.321533,other,1,1,0,0
2,1,0.1,0.343584,-1,117,70,0.000449,-0.875978,AC,944,...,0,200.0,0,INTERNET,1.410795,windows,0,2,0,0
3,0,0.5,0.121816,-1,56,20,0.008242,11.078060,AA,2495,...,0,200.0,0,INTERNET,8.095574,other,1,1,0,0
4,0,0.1,0.103161,-1,21,60,0.043340,22.547734,AA,1728,...,1,200.0,0,INTERNET,22.010295,windows,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0.9,0.296803,-1,99,50,0.029718,-0.660974,AC,940,...,0,200.0,0,INTERNET,13.295473,windows,0,1,0,7
999996,0,0.5,0.087615,-1,162,60,2.110916,-1.465277,AC,942,...,0,500.0,0,INTERNET,5.904327,windows,0,1,0,7
999997,0,0.8,0.740167,138,12,30,18.470612,-1.118478,AB,353,...,0,200.0,0,INTERNET,4.184912,windows,0,1,0,7
999998,0,0.9,0.377040,-1,38,30,0.022148,-1.170191,AB,1243,...,0,1500.0,0,INTERNET,7.263938,linux,1,1,0,7


In [121]:
df4['fraud_bool'].value_counts()

fraud_bool
0    988970
1     11030
Name: count, dtype: int64

In [124]:
df4_6m=df4[df4['month']<=5]

In [125]:
# 查看customer_age大于40并且小于80的数量
age50_more=df4_6m[df4_6m['customer_age']>=50]

# 查看customer_age小于50的数量
age50_less=df4_6m[df4_6m['customer_age']<=40]

age50_less.shape[0]/(age50_more.shape[0]+age50_less.shape[0])

0.49403292506672575

In [126]:
# 查看age50_more和age50_less的欺诈率（即fraud_bool=1的比例）
prevalence_major=age50_more[age50_more['fraud_bool']==1].shape[0]/age50_more.shape[0]
prevalence_minor=age50_less[age50_less['fraud_bool']==1].shape[0]/age50_less.shape[0]

prevalence_major, prevalence_minor

(0.018283036521766327, 0.003638169010700968)

In [127]:
df4_2m=df4[df4['month']>5]

# 查看customer_age大于40并且小于80的数量
age50_more=df4_2m[df4_2m['customer_age']>=50]

# 查看customer_age小于50的数量
age50_less=df4_2m[df4_2m['customer_age']<=40]

age50_less.shape[0]/(age50_more.shape[0]+age50_less.shape[0])

0.49467460028468335

In [128]:
# 查看age50_more和age50_less的欺诈率（即fraud_bool=1的比例）
prevalence_major=age50_more[age50_more['fraud_bool']==1].shape[0]/age50_more.shape[0]
prevalence_minor=age50_less[age50_less['fraud_bool']==1].shape[0]/age50_less.shape[0]

prevalence_major, prevalence_minor

(0.011128124923233134, 0.010815648813457243)

In [129]:
df4_2m

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
758328,0,0.5,0.830797,-1,395,20,0.017043,38.302270,AA,911,...,0,200.0,0,INTERNET,10.636646,windows,0,1,0,6
758329,0,0.9,0.499809,-1,36,20,0.002148,-0.550935,AC,613,...,0,200.0,0,INTERNET,5.908758,other,0,1,0,6
758330,0,0.3,0.822515,-1,195,50,0.029686,-1.308534,AC,1576,...,0,1000.0,0,INTERNET,7.996862,other,1,1,0,6
758331,0,0.6,0.114865,-1,147,70,0.030274,-1.214061,AB,2955,...,0,500.0,0,INTERNET,4.004354,windows,0,1,0,6
758332,0,0.3,0.244592,28,42,50,0.005425,50.884248,AA,1060,...,1,200.0,0,INTERNET,26.912111,macintosh,0,2,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0.9,0.296803,-1,99,50,0.029718,-0.660974,AC,940,...,0,200.0,0,INTERNET,13.295473,windows,0,1,0,7
999996,0,0.5,0.087615,-1,162,60,2.110916,-1.465277,AC,942,...,0,500.0,0,INTERNET,5.904327,windows,0,1,0,7
999997,0,0.8,0.740167,138,12,30,18.470612,-1.118478,AB,353,...,0,200.0,0,INTERNET,4.184912,windows,0,1,0,7
999998,0,0.9,0.377040,-1,38,30,0.022148,-1.170191,AB,1243,...,0,1500.0,0,INTERNET,7.263938,linux,1,1,0,7
